In [ ]:
!pip install pygame
!pip install rich

In [3]:
from IPython.display import Audio, display
from random import choice
import pygame
import time
from rich import print
from rich.console import Console
from rich.panel import Panel


# define rooms and items
door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door"
}

door_c = {
    "name": "door c",
    "type": "door"
}

door_d = {
    "name": "door d",
    "type": "door"
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b

}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c

}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d

}

couch = {
    "name": "couch",
    "type": "furniture",
}

piano = {
    "name": "piano",
    "type": "furniture",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture"
}

double_bed = {
    "name": "double bed",
    "type": "furniture"
}

dresser = {
    "name": "dresser",
    "type": "furniture"
}

dining_table = {
    "name": "dining table",
    "type": "furniture"
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room"
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room"
}

living_room = {
    "name": "living room",
    "type": "room"
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a,door_b,door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "bedroom 1": [queen_bed,door_a,door_b,door_c],
    "bedroom 2": [door_b,double_bed,dresser],
    "living room": [dining_table,door_d,door_c],
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_c],
    "dresser": [key_d],
    "outside": [door_d],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_2,bedroom_1],
    "door c": [living_room, bedroom_1],
    "door d": [living_room,outside]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [4]:

pygame.mixer.init()

console = Console()

def play_music(path, loop=False):
    pygame.mixer.music.stop()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play(-1 if loop else 0)

def stop_music():
    pygame.mixer.music.stop()


class UserError(Exception):
    pass

class User_input:
    def __init__(self, intended_action):
        self.intended_action = intended_action.strip()
        if intended_action not in ["explore", "examine", "head", "tail", "yes", "no"]:
            raise UserError

def linebreak():
    console.rule("[green]Room Transition[/green]")

def start_game():
    play_music("caves-of-dawn-10376.mp3", loop=True)
    console.print(Panel.fit("ESCAPE ROOM", style="bold white on red", title="Game Start"))
    console.print("[bold magenta]You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW![/bold magenta]")
    play_room(game_state["current_room"])

def play_room(room):
    game_state["current_room"] = room

    if game_state["current_room"] == game_state["target_room"]:
        stop_music()
        play_music("dooropened-103851.mp3")
        console.print("[bold green]Congrats! You escaped the room![/bold green]")
    else:
        console.print("[cyan]You are now in[/cyan] " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'? ").strip()

        try:
            action = User_input(intended_action)
            if action.intended_action == "explore":
                explore_room(room)
                play_room(room)
            elif action.intended_action == "examine":
                examine_item(input("What would you like to examine? ").strip())
        except UserError:
            console.print(f"[bold red]{UserError}: you have to type examine or explore[/bold red]")
            play_room(room)

        linebreak()

def explore_room(room):
    items = [i["name"] for i in object_relations[room["name"]]]
    console.print("[yellow]You explore the room. This is[/yellow] " + room["name"] + ". [yellow]You find[/yellow] " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if not current_room == room:
            return room

def examine_item(item_name):
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            output = "[bold]You examine " + item_name + ".[/bold] "
            if item["type"] == "door":
                have_key = any(key["target"] == item for key in game_state["keys_collected"])
                if have_key:
                    console.print("[green]To get to the next room, you must win in heads or tails.[/green]")
                    output += " You unlock the door."

                    while True:
                        user_choice = input("What do you choose? head or tail ").strip()
                        try:
                            u_choice = User_input(user_choice)
                            if u_choice.intended_action in ["head", "tail"]:
                                side_coin = choice(["head", "tail"])
                                if u_choice.intended_action == side_coin:
                                    console.print("[bold yellow]Congratulations! It was " + side_coin + "[/bold yellow]")
                                    stop_music()
                                    play_music("dooropened-103851.mp3")
                                    time.sleep(2)
                                    play_music("caves-of-dawn-10376.mp3", loop=True)
                                    next_room = get_next_room_of_door(item, current_room)
                                    break
                                else:
                                    console.print("[red]Sorry! It was " + side_coin + "[/red]")
                                    console.print("Try again, choose between head or tail.")
                            else:
                                console.print("[red]Invalid input. Try again.[/red]")
                        except UserError:
                            console.print(f"[red]{UserError}: You have to choose between head or tail.[/red]")
                else:
                    output += "[yellow]It is locked but you don't have the key.[/yellow]"
            else:
                if item["name"] in object_relations and len(object_relations[item["name"]]) > 0:
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    stop_music()
                    play_music("llaveskeys-338166.mp3")
                    time.sleep(2)
                    play_music("caves-of-dawn-10376.mp3", loop=True)
                    output += "[green]You find " + item_found["name"] + ".[/green]"
                else:
                    output += "There isn't anything interesting about it."
            console.print(output)
            break

    if output is None:
        console.print("[red]The item you requested is not found in the current room.[/red]")

    if next_room and input("Do you want to go to the next room? Enter 'yes' or 'no': ").strip() == 'yes':
        play_room(next_room)
    else:
        play_room(current_room)



"""
###NEW CODE ############
    if(next_room):
      yes_no_input = input("Do you want to go to the next room? Enter 'yes' or 'no'").strip()
      while True:
          try:
              yes_no_object = User_input(yes_no_input)
              if yes_no_object.intended_action == "yes":
                  play_room(next_room)
                  break
              elif yes_no_object.intended_action == "no":
                  print("You’ll stay in the current room")
                  play_room(current_room)
                  break
          except UserError:
              print(f"{UserError}: you have to type yes or no")
              yes_no_input = input("Enet yes or no: ").strip()


"""

'\n###NEW CODE ############\n    if(next_room):\n      yes_no_input = input("Do you want to go to the next room? Enter \'yes\' or \'no\'").strip()\n      while True:\n          try:\n              yes_no_object = User_input(yes_no_input)\n              if yes_no_object.intended_action == "yes":\n                  play_room(next_room)\n                  break\n              elif yes_no_object.intended_action == "no":\n                  print("You’ll stay in the current room")\n                  play_room(current_room)\n                  break\n          except UserError:\n              print(f"{UserError}: you have to type yes or no")\n              yes_no_input = input("Enet yes or no: ").strip()\n\n\n'

In [5]:
game_state = INIT_GAME_STATE.copy()

start_game()

╭─ Game Start ─╮
│ ESCAPE ROOM  │
╰──────────────╯

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. 
You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and 
you must get out of the house, NOW!

You are now in game room

You explore the room. This is game room. You find couch, piano, door a

You are now in game room

...

───────────────────────────────────────────────── Room Transition ─────────────────────────────────────────────────

───────────────────────────────────────────────── Room Transition ─────────────────────────────────────────────────

───────────────────────────────────────────────── Room Transition ─────────────────────────────────────────────────

───────────────────────────────────────────────── Room Transition ─────────────────────────────────────────────────

───────────────────────────────────────────────── Room Transition ─────────────────────────────────────────────────